# Question 1:  Pose Graph Optimization for 1D SLAM

A solved example for 1D SLAM which optimizes for pose variables using weighted least squares method (Gauss Newton) has been explained in the class. You can view it at `examples/1D-SLAM.ipynb`. Your first task is to code this from scratch.

For this section, you have to calculate Jacobian analytically yourself and use it. However, you can check how correct it is using `jax`'s `jacobian`. Its usage is explained in the supplementary notebook (`examples/`).

## How to get started?

1. Go through the example and understand the optimization procedure.
2. Generate the data as in the example.
3. Write the derivatives of the function with respect to the constraints
4. Create the final jacobian and write the update steps using Gauss Newton

This tiny example will help make sense of the next question, as it's largely just an extension of this.

Finally, plot the loop closure constraint, poses, and error at every iteration as well. Since this is a one dimensional example, you can just assume x = y.

In [1]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
import jax.numpy as jnp
from jax import jacfwd
%matplotlib inline

In [2]:
def fux(X,U):
    funX = []
    nodes = X.shape[0]
    for i in range(nodes):
        # Control Constraints for 0-4: x(i) + u(i) - x(i+1)
        # Loop Constraints : x(0) + (Loop Constraint) - x(len(x))
        # Last line: x(0)
        if(i != nodes-1):
            val = X[i] + U[i] - X[i+1]
        else:
            val = X[0] + U[i] - X[nodes-1]
        funX.append(val)
    funX.append(X[0])
    return funX

In [7]:
def frobNorm(P1, P2, str1="mat1", str2="mat2"):
    jnp.set_printoptions(suppress=True)
    val = jnp.linalg.norm(P1 - P2, 'fro')
    print(f"Frobenius norm between {str1} and {str2} is: {val}")

In [3]:
Ugt = jnp.array([1.0,1.0,1.0,-3.0,0.0])
Xgt = jnp.array([0.0,1.0,2.0,3.0,0.0])

Xobs = jnp.array([0.0,1.1,2.1,3.2,0.5])
Uobs = jnp.array([1.1,1.0,1.1,-2.7,0.0])

OMG = jnp.array([[100.0,0,0,0,0,0],
           [0,100.0,0,0,0,0],
           [0,0,100.0,0,0,0],
           [0,0,0,100.0,0,0],
           [0,0,0,0,100.0,0],
           [0,0,0,0,0,1000.0]])
J = jnp.array([[1,-1,0,0,0],
               [0,1,-1,0,0],
               [0,0,1,-1,0],
               [0,0,0,1,-1],
               [1,0,0,0,-1],
               [1,0,0,0,0]])
Error = []

In [5]:
for i in range(15):
    funX = fux(Xobs,Uobs)
#     b = (jnp.matmul((J.T),(OMG.T)))
#     #print(b.shape)
    b = jnp.matmul(jnp.matmul((J.T),(OMG.T)),jnp.array(funX))
    H = jnp.matmul(jnp.matmul((J.T),(OMG)),J)
    deltaX = jnp.matmul(jnp.linalg.pinv(H),b)
    Xobs = Xobs - deltaX
#     err = get_error(pos, odometer, weights)
    print("Updated Xobs = "+str(Xobs))
    print(
    
f = lambda Xobs: fux(Xobs,Uobs) # Now f is just a function of variable to be differentiated
J_a = jacfwd(f)
J_ac = J_a(Xobs)

Updated Xobs = [-3.5764103e-10  1.0000000e+00  1.9000000e+00  2.9000001e+00
  9.9999964e-02]
Updated Xobs = [-2.3842642e-10  1.0000000e+00  1.9000000e+00  2.9000001e+00
  1.0000000e-01]
Updated Xobs = [-3.278278e-10  1.000000e+00  1.900000e+00  2.900000e+00  1.000000e-01]
Updated Xobs = [-3.2782868e-10  1.0000000e+00  1.9000000e+00  2.9000001e+00
  1.0000000e-01]
Updated Xobs = [-3.278278e-10  1.000000e+00  1.900000e+00  2.900000e+00  1.000000e-01]
Updated Xobs = [-3.2782868e-10  1.0000000e+00  1.9000000e+00  2.9000001e+00
  1.0000000e-01]
Updated Xobs = [-3.278278e-10  1.000000e+00  1.900000e+00  2.900000e+00  1.000000e-01]
Updated Xobs = [-3.2782868e-10  1.0000000e+00  1.9000000e+00  2.9000001e+00
  1.0000000e-01]
Updated Xobs = [-3.278278e-10  1.000000e+00  1.900000e+00  2.900000e+00  1.000000e-01]
Updated Xobs = [-3.2782868e-10  1.0000000e+00  1.9000000e+00  2.9000001e+00
  1.0000000e-01]
Updated Xobs = [-3.278278e-10  1.000000e+00  1.900000e+00  2.900000e+00  1.000000e-01]
Updated

In [6]:
if jnp.allclose(J_ac, J):
    print("Matches")
else:
    print("Doesn't Match")

Matches
